In [9]:
# Importing all necessary libraries
%load_ext autoreload
%autoreload 2

# internal packages
import os
from collections import Counter, OrderedDict

# external packages
import torch
import torchvision
import numpy as np
import sklearn
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score, confusion_matrix
from sklearn.decomposition import PCA
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd

# util functions
from util.util import *

# dataset functions
from dataset import load_util

# autoencoder
from models.autoencoder.conv_ae import ConvAE
from models.simclr.simclr import *
from models.simclr.transforms import *
from models.rotnet.rotnet import *
from models.rotnet.IDEC import *
from models.rotnet.custom_stl10 import *
from cluster_accuracy import cluster_accuracy
from models.simclr.custom_stl10 import SimCLRSTL10

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
print("Versions:")
print(f"torch: {torch.__version__}")
print(f"torchvision: {torchvision.__version__}")
print(f"numpy: {np.__version__}",)
print(f"scikit-learn: {sklearn.__version__}")

device = detect_device()
print("Using device: ", device)

Versions:
torch: 1.8.1+cu111
torchvision: 0.9.1+cu111
numpy: 1.19.5
scikit-learn: 0.24.1
Using device:  cuda


## Preparation

In [4]:
# specify learning params
batch_size = 128
learning_rate = 0.1
epochs = 100

# training

train = True

In [18]:
cifar_test_data = load_util.load_custom_cifar('./data', download=False, train=False, data_percent=1.0, for_model='SimCLR', transforms=False)
cifar_testloader = torch.utils.data.DataLoader(test_data,
                                          batch_size=128,
                                          shuffle=True,
                                          drop_last=True)

In [19]:
cifar_colors_classes = {i: color_class for i, color_class in zip(range(len(test_data.classes)), test_data.classes)}

In [29]:
stl10_test_data = SimCLRSTL10(data_percent=1.0, train=False)
stl10_testloader = torch.utils.data.DataLoader(stl10_test_data,
                                         batch_size=128,
                                         shuffle=True,
                                         drop_last=True)

In [12]:
stl10_colors_classes = {i: color_class for i, color_class in zip(range(len(stl10_test_data.classes)), stl10_test_data.classes)}

In [31]:
def get_nmis_and_cas_10_runs(model_name, colors_classes, device, dataset='CIFAR'):
    nmis = {}
    cas = {}
    
    testloader = cifar_testloader if 'CIFAR' in dataset else stl10_testloader

    for i in range(10):
        name = f'{model_name}_{i}.pth'
        print(name)
        
        model = load_model(name, device, torch.rand(size=(4, 12288)))
        nmis[i] = []
        cas[i] = []

        for k in range(10):
            labels, kmeans, nmi, _, _ = compute_nmi_and_pca(model, name, colors_classes, device, testloader)
            nmis[i].append(nmi)

            ca = cluster_accuracy(labels, kmeans.labels_)
            cas[i].append(ca)
            print(f'Run: {k}\n{name}:\nNMI:{nmi}\nCA:{ca}\n')
            del labels
            del kmeans
    return nmis, cas

In [8]:
def print_total_mean_and_var(metrics_name, df):
    models_means = df.mean()
    models_stds = df.std()

    total_mean = models_means.mean()
    total_std = models_stds.std()
    
    print(f'{metrics_name} mean: {total_mean:.4f}')
    print(f'{metrics_name} variance: {total_std:.4f}')

## RotNet

### CIFAR : Pretraining

In [110]:
path_nmis = 'trained_models/RotNet/CIFAR/nmis.csv'
path_cas = 'trained_models/RotNet/CIFAR/cas.csv'

Uncomment this part to generate new NMI and cluster accuracies

In [ ]:
# rotnet_nmis_cifar, rotnet_cas_cifar = get_nmis_and_cas_10_runs('RotNet\CIFAR\pretrained_RotNet', colors_classes, device)

In [ ]:
# df_rotnet_nmis_cifar = pd.DataFrame(rotnet_nmis_cifar)
# df_rotnet_cas_cifar = pd.DataFrame(rotnet_cas_cifar)

In [74]:
# df_rotnet_nmis_cifar.to_csv(path_nmis)
# df_rotnet_cas_cifar.to_csv(path_cas)

This code loads existing dataframes from specified paths

In [111]:
df_rotnet_nmis_cifar = pd.read_csv(path_nmis, index_col=0)
df_rotnet_cas_cifar = pd.read_csv(path_cas, index_col=0)

In [112]:
print_total_mean_and_var('NMI', df_rotnet_nmis_cifar)

NMI mean: 0.3506
NMI variance: 0.0017


In [113]:
print_total_mean_and_var('Cluser accuracy', df_rotnet_cas_cifar)

Cluser accuracy mean: 0.4357
Cluser accuracy variance: 0.0051


### CIFAR: DEC

In [114]:
path_nmis = 'trained_models/RotNet/CIFAR/nmis_dec.csv'
path_cas = 'trained_models/RotNet/CIFAR/cas_dec.csv'

Uncomment this part to generate new NMI and cluster accuracies

In [ ]:
# rotnet_nmis_cifar_dec, rotnet_cas_cifar_dec = get_nmis_and_cas_10_runs('RotNet\CIFAR\DEC_RotNet', colors_classes, device)

In [ ]:
# df_rotnet_nmis_cifar_dec = pd.DataFrame(rotnet_nmis_cifar_dec)
# df_rotnet_cas_cifar_dec = pd.DataFrame(rotnet_cas_cifar_dec)

In [105]:
# df_rotnet_nmis_cifar_dec.to_csv(path_nmis)
# df_rotnet_cas_cifar_dec.to_csv(path_cas)

This code loads existing dataframes from specified paths

In [115]:
df_rotnet_nmis_cifar_dec = pd.read_csv(path_nmis, index_col=0)
df_rotnet_cas_cifar_dec = pd.read_csv(path_cas, index_col=0)

In [117]:
print_total_mean_and_var('NMI', df_rotnet_nmis_cifar_dec)

NMI mean: 0.1042
NMI variance: 0.0001


In [118]:
print_total_mean_and_var('Cluser accuracy', df_rotnet_cas_cifar_dec)

Cluser accuracy mean: 0.2065
Cluser accuracy variance: 0.0002


### STL10: Pretraining

In [14]:
path_nmis = 'trained_models/RotNet/STL10/nmis.csv'
path_cas = 'trained_models/RotNet/STL10/cas.csv'

Uncomment this part to generate new NMI and cluster accuracies

In [ ]:
rotnet_nmis_stl10, rotnet_cas_stl10 = get_nmis_and_cas_10_runs('RotNet\STL10\pretrained_RotNet_STL10', stl10_colors_classes, device, dataset='STL10')

In [33]:
df_rotnet_nmis_stl10 = pd.DataFrame(rotnet_nmis_stl10)
df_rotnet_cas_stl10 = pd.DataFrame(rotnet_cas_stl10)

In [34]:
df_rotnet_nmis_stl10.to_csv(path_nmis)
df_rotnet_cas_stl10.to_csv(path_cas)

In [35]:
df_rotnet_nmis_stl10 = pd.read_csv(path_nmis, index_col=0)
df_rotnet_cas_stl10 = pd.read_csv(path_cas, index_col=0)

In [36]:
print_total_mean_and_var('NMI', df_rotnet_nmis_stl10)

NMI mean: 0.3357
NMI variance: 0.0032


In [37]:
print_total_mean_and_var('Cluser accuracy', df_rotnet_cas_stl10)

Cluser accuracy mean: 0.3641
Cluser accuracy variance: 0.0076


### STL10: DEC

In [39]:
path_nmis = 'trained_models/RotNet/STL10/nmis_dec.csv'
path_cas = 'trained_models/RotNet/STL10/cas_dec.csv'

Uncomment this part to generate new NMI and cluster accuracies

In [ ]:
# rotnet_nmis_stl10_dec, rotnet_cas_stl10_dec = get_nmis_and_cas_10_runs('RotNet\STL10\DEC_RotNet_STL10', stl10_colors_classes, device, dataset='STL10')

In [41]:
# df_rotnet_nmis_stl10_dec = pd.DataFrame(rotnet_nmis_stl10_dec)
# df_rotnet_cas_stl10_dec = pd.DataFrame(rotnet_cas_stl10_dec)

In [42]:
# df_rotnet_nmis_stl10_dec.to_csv(path_nmis)
# df_rotnet_cas_stl10_dec.to_csv(path_cas)

In [43]:
df_rotnet_nmis_stl10_dec = pd.read_csv(path_nmis, index_col=0)
df_rotnet_cas_stl10_dec = pd.read_csv(path_cas, index_col=0)

In [44]:
print_total_mean_and_var('NMI', df_rotnet_nmis_stl10_dec)

NMI mean: 0.1292
NMI variance: 0.0003


In [45]:
print_total_mean_and_var('Cluser accuracy', df_rotnet_cas_stl10_dec)

Cluser accuracy mean: 0.1967
Cluser accuracy variance: 0.0003


## SimCLR

### CIFAR: Pretraining

In [130]:
path_nmis = 'trained_models/SimCLR/CIFAR/nmis.csv'
path_cas = 'trained_models/SimCLR/CIFAR/cas.csv'

Uncomment this part to generate new NMI and cluster accuracies

In [ ]:
# simclr_nmis_cifar, simclr_cas_cifar = get_nmis_and_cas_10_runs('SimCLR\CIFAR\pretrained_SimCLR', colors_classes, device)

In [24]:
# df_simclr_nmis_cifar = pd.DataFrame(simclr_nmis_cifar)
# df_simclr_cas_cifar = pd.DataFrame(simclr_cas_cifar)

In [134]:
# df_simclr_nmis_cifar.to_csv(path_nmis)
# df_simclr_cas_cifar.to_csv(path_cas)

In [132]:
df_simclr_nmis_cifar = pd.read_csv(path_nmis, index_col=0)
df_simclr_cas_cifar = pd.read_csv(path_cas, index_col=0)

In [135]:
print_total_mean_and_var('NMI', df_simclr_nmis_cifar)

NMI mean: 0.4724
NMI variance: 0.0033


In [136]:
print_total_mean_and_var('Cluser accuracy', df_simclr_cas_cifar)

Cluser accuracy mean: 0.5904
Cluser accuracy variance: 0.0090


In [98]:
columns = [f'Run {i}' for i in range(10)]

In [133]:
df_simclr_nmis_cifar.columns = columns
df_simclr_cas_cifar.columns = columns

### CIFAR: IDEC

In [137]:
path_nmis = 'trained_models/SimCLR/CIFAR/nmis_idec.csv'
path_cas = 'trained_models/SimCLR/CIFAR/cas_idec.csv'

Uncomment this part to generate new NMI and cluster accuracies

In [ ]:
# simclr_nmis_cifar_idec, simclr_cas_cifar_idec = get_nmis_and_cas_10_runs('SimCLR\CIFAR\IDEC_SimCLR', colors_classes, device)

In [27]:
# df_simclr_nmis_cifar_idec = pd.DataFrame(simclr_nmis_cifar_idec)
# df_simclr_cas_cifar_idec = pd.DataFrame(simclr_cas_cifar_idec)

In [141]:
# df_simclr_nmis_cifar_idec.to_csv(path_nmis)
# df_simclr_cas_cifar_idec.to_csv(path_cas)

In [138]:
df_simclr_nmis_cifar_idec = pd.read_csv(path_nmis, index_col=0)
df_simclr_cas_cifar_idec = pd.read_csv(path_cas, index_col=0)

In [142]:
print_total_mean_and_var('NMI', df_simclr_nmis_cifar_idec)

NMI mean: 0.5569
NMI variance: 0.0001


In [143]:
print_total_mean_and_var('Cluser accuracy', df_simclr_cas_cifar_idec)

Cluser accuracy mean: 0.6576
Cluser accuracy variance: 0.0000


### STL10: Pretraining

In [46]:
path_nmis = 'trained_models/SimCLR/STL10/nmis.csv'
path_cas = 'trained_models/SimCLR/STL10/cas.csv'

Uncomment this part to generate new NMI and cluster accuracies

In [ ]:
simclr_nmis_stl10, simclr_cas_stl10 = get_nmis_and_cas_10_runs('SimCLR\STL10\pretrained_SimCLR_STL10', stl10_colors_classes, device, dataset='STL10')

In [48]:
df_simclr_nmis_stl10 = pd.DataFrame(simclr_nmis_stl10)
df_simclr_cas_stl0 = pd.DataFrame(simclr_cas_stl10)

In [49]:
df_simclr_nmis_stl10.to_csv(path_nmis)
df_simclr_cas_stl0.to_csv(path_cas)

In [50]:
df_simclr_nmis_stl10 = pd.read_csv(path_nmis, index_col=0)
df_simclr_cas_stl0 = pd.read_csv(path_cas, index_col=0)

In [51]:
print_total_mean_and_var('NMI', df_simclr_nmis_stl10)

NMI mean: 0.3502
NMI variance: 0.0015


In [52]:
print_total_mean_and_var('Cluser accuracy', df_simclr_cas_stl0)

Cluser accuracy mean: 0.4114
Cluser accuracy variance: 0.0051


### STL10: IDEC

In [55]:
path_nmis_dec = 'trained_models/SimCLR/STL10/nmis_idec.csv'
path_cas_dec = 'trained_models/SimCLR/STL10/cas_idec.csv'

Uncomment this part to generate new NMI and cluster accuracies

In [58]:
simclr_nmis_stl10_idec, simclr_cas_stl10_idec = get_nmis_and_cas_10_runs('SimCLR\STL10\IDEC_SimCLR', stl10_colors_classes, device, dataset='STL10')

SimCLR\STL10\IDEC_SimCLR_0.pth
Run: 0
SimCLR\STL10\IDEC_SimCLR_0.pth:
NMI:0.3869628444149622
CA:0.40940020161290325

Run: 1
SimCLR\STL10\IDEC_SimCLR_0.pth:
NMI:0.3879816584306516
CA:0.4592993951612903

Run: 2
SimCLR\STL10\IDEC_SimCLR_0.pth:
NMI:0.38804349746831
CA:0.45955141129032256

Run: 3
SimCLR\STL10\IDEC_SimCLR_0.pth:
NMI:0.38824743891354596
CA:0.4609375

Run: 4
SimCLR\STL10\IDEC_SimCLR_0.pth:
NMI:0.38797855102524814
CA:0.45904737903225806

Run: 5
SimCLR\STL10\IDEC_SimCLR_0.pth:
NMI:0.389371483921124
CA:0.46018145161290325

Run: 6
SimCLR\STL10\IDEC_SimCLR_0.pth:
NMI:0.38789969893711596
CA:0.46118951612903225

Run: 7
SimCLR\STL10\IDEC_SimCLR_0.pth:
NMI:0.38762481632268164
CA:0.4598034274193548

Run: 8
SimCLR\STL10\IDEC_SimCLR_0.pth:
NMI:0.3885151264576402
CA:0.4604334677419355



KeyboardInterrupt: 

In [ ]:
df_simclr_nmis_stl10_idec = pd.DataFrame(simclr_nmis_stl10_idec)
df_simclr_cas_stl10_idec = pd.DataFrame(simclr_cas_stl10_idec)

In [ ]:
df_simclr_nmis_stl10_idec.to_csv(path_nmis)
df_simclr_cas_stl10_idec.to_csv(path_cas)

In [ ]:
df_simclr_nmis_stl10_idec = pd.read_csv(path_nmis, index_col=0)
df_simclr_cas_stl10_idec = pd.read_csv(path_cas, index_col=0)

In [ ]:
print_total_mean_and_var('NMI', df_simclr_nmis_stl10_idec)

In [ ]:
print_total_mean_and_var('Cluser accuracy', df_simclr_cas_stl10_idec)